In [10]:
#載入套件
import os
import pandas as pd
import numpy as np


#讀取訓練與測試資料 way1
data_path = 'C:\\python-training\\007'
f_app_train = os.path.join(data_path, 'house_train.csv.gz')
app_train = pd.read_csv(f_app_train)
app_train.shape

(1460, 81)

In [16]:
#讀取訓練與測試資料 way2
#直接讀取 壓縮檔案時 需要在資料夾後多加 \\
data_path = 'C:\\python-training\\007\\'
df_train = pd.read_csv(data_path + 'house_train.csv.gz')
df_test = pd.read_csv(data_path + 'house_test.csv.gz')
df_train.shape

(1460, 81)

In [17]:
# log1p = log（x+1）數據壓縮到了一個區間，與數據的標準類似
# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程

train_Y = np.log1p(df_train['SalePrice'])
ids = df_test['Id']

'''
1）drop函數的使用：刪除行、刪除列
print frame.drop(['a'])
print frame.drop(['Ohio'], axis = 1)
drop函數默認刪除行，列需要加axis = 1
'''
'''
concat中有很多參數可以調整,合併成你想要的數據形式.
axis (合併方向) 沒寫就是垂直合併 
'''

df_train = df_train.drop(['Id', 'SalePrice'] , axis=1)
df_test = df_test.drop(['Id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head() #前五rows

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [18]:
# 秀出資料欄位的類型, 與對應的數量
# df.dtypes : 轉成以欄位為 index, 類別(type)為 value 的 DataFrame
# .reset_index() : 預設是將原本的 index 轉成一個新的欄位, 如果不須保留 index, 則通常會寫成 .reset_index(drop=True)

'''
df 是整個 dataframe，df.dtypes 是每個 columns 的型態所組成的 df，他應該會長這樣：

    0      |   1
--------------------
Col #1 | int
Col #2 | int
Col #3 | float
加上 .reset_index() 只是把每一列的索引名稱重新調整而已，避免一些奇怪的 index  出現。

df.columns = .. 這一行是把欄位改名，結果應該變成這樣：
 count | type
--------------------
Col #1 | int
Col #2 | int
Col #3 | float

groupby('type').agg('count') 這一行是利用 type 欄位分組（int 的一組、float 的一組），
計算出每一組的個數，所以結果如下：
count | type
--------------------
        2 | int
        1 | float
接著，我們來看圖二，他是延續剛剛的這個結果：
count | type
--------------------
Col #1 | int
Col #2 | int
Col #3 | float
.groupby('count').agg('count') 這一行改成利用 count 欄位分組（Col #1 的一組、Col #2 的一組 ...），計算出每一組的個數，所以結果如下：
count | type
--------------------
Col #1 | 1
Col #2 | 1
Col #3 | 1
'''

dtype_df = df.dtypes.reset_index() 
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,25
1,float64,11
2,object,43


In [20]:
# 確定只有 int64, float64, object 三種類型後對欄位名稱執行迴圈, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []

# .dtypes(欄位類型), .columns(欄位名稱) 是 DataFrame 提供的兩個方法, 這裡順便展示一下 for 與 zip 搭配的用法
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
'''        
# 這邊採用的寫法稱為 f-string, 是 Python 3.6.2 以後版本才出現的
# 如果無法執行, 則需要更新到這個版本之後, 或自行將程式改寫為 str.format 形式
# 改寫方式可以參考 https://blog.louie.lu/2017/08/08/outdate-python-string-format-and-fstring/
f'字串插值
text = 'world'
print(f'Hello, {text}')
參考：https://blog.techbridge.cc/2019/05/03/how-to-use-python-string-format-method/
'''  

print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

25 Integer Features : ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

11 Float Features : ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

43 Object Features : ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fi

In [24]:
#這樣就可以單獨秀出特定類型的欄位集合, 方便做後續的特徵工程處理
df[float_features].head()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0
1,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0
2,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0
3,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0
4,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0


In [23]:
df[object_features].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [26]:
'''
作業1
試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?

操作如下，只有 object 的 mean() 與 max() 會有問題，原因是文字無法進行平均或比較大小
但我們可以發現，object 仍有部分欄位可傳回最大值，可以知道 object 欄位超過一定長度時會當成字串，此時會以字典順序做大小比較
'''

# 整數 (int) 特徵取平均 (mean)
df[int_features].mean()

MSSubClass          57.137718
LotArea          10168.114080
OverallQual          6.089072
OverallCond          5.564577
YearBuilt         1971.312778
YearRemodAdd      1984.264474
1stFlrSF          1159.581706
2ndFlrSF           336.483727
LowQualFinSF         4.694416
GrLivArea         1500.759849
FullBath             1.568003
HalfBath             0.380267
BedroomAbvGr         2.860226
KitchenAbvGr         1.044536
TotRmsAbvGrd         6.451524
Fireplaces           0.597122
WoodDeckSF          93.709832
OpenPorchSF         47.486811
EnclosedPorch       23.098321
3SsnPorch            2.602261
ScreenPorch         16.062350
PoolArea             2.251799
MiscVal             50.825968
MoSold               6.213087
YrSold            2007.792737
dtype: float64

In [27]:
# 整數 (int) 特徵取最大值 (max)
df[int_features].max()

MSSubClass          190
LotArea          215245
OverallQual          10
OverallCond           9
YearBuilt          2010
YearRemodAdd       2010
1stFlrSF           5095
2ndFlrSF           2065
LowQualFinSF       1064
GrLivArea          5642
FullBath              4
HalfBath              2
BedroomAbvGr          8
KitchenAbvGr          3
TotRmsAbvGrd         15
Fireplaces            4
WoodDeckSF         1424
OpenPorchSF         742
EnclosedPorch      1012
3SsnPorch           508
ScreenPorch         576
PoolArea            800
MiscVal           17000
MoSold               12
YrSold             2010
dtype: int64

In [28]:
# 整數 (int) 特徵取相異值 (nunique)
df[int_features].nunique()

MSSubClass         16
LotArea          1951
OverallQual        10
OverallCond         9
YearBuilt         118
YearRemodAdd       61
1stFlrSF         1083
2ndFlrSF          635
LowQualFinSF       36
GrLivArea        1292
FullBath            5
HalfBath            3
BedroomAbvGr        8
KitchenAbvGr        4
TotRmsAbvGrd       14
Fireplaces          5
WoodDeckSF        379
OpenPorchSF       252
EnclosedPorch     183
3SsnPorch          31
ScreenPorch       121
PoolArea           14
MiscVal            38
MoSold             12
YrSold              5
dtype: int64

In [29]:
# 類別 (object) 特徵取平均 (mean)
df[object_features].mean()

Series([], dtype: float64)

In [30]:
# 類別 (object) 特徵取最大值 (max)
df[object_features].max()

Series([], dtype: float64)

In [31]:
# 類別 (object) 特徵取相異值 (nunique)
df[object_features].nunique()

MSZoning          5
Street            2
Alley             2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         5
CentralAir        2
Electrical        5
KitchenQual       4
Functional        7
FireplaceQu       5
GarageType        6
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
PoolQC            3
Fence             4
MiscFeature       4
SaleType          9
SaleCondition     6
dtype: int64

In [ ]:
'''
作業2
思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?
所以三大類特徵中，哪一大類處理起來應該最複雜?

舉例 : 複數 (a+bi)，因為需要兩個數字表示，雖然概念上是數值，但因為無法比較大小，因此只能歸類為"類別型欄位"
不論其他更奇怪的例子，都可以歸類為類別型欄位，因此類別型欄位應該最難處理
''''''